In [9]:
import csv
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import convolve2d
from threading import Thread

In [10]:

def pxToMicrometers(pixels, scaleLength, unit):
    return (pixels/scaleLength)*unit

def randomizeCoordinates(edge):
    height, width = edge.shape
    return (np.random.randint(height),np.random.randint(height),np.random.randint(width),np.random.randint(width))



def calculate(imgName, scaleLength, unit, iterations=500, images=True):
   
    # Load the image
    img = Image.open(f'./tests/{imgName}')

    # Convert the image to grayscale
    gray = img.convert('L')

    # Convert the grayscale image to a NumPy array
    img_array = np.array(gray)
    values = [0 for _ in range(iterations)]
    # Define the Sobel filter
    sobel_filter = np.array([[-1], [0], [1]])
    # Apply the Sobel filter to the image
    edge_x = np.abs(convolve2d(img_array, sobel_filter, mode="same"))
    edge_y = np.abs(convolve2d(img_array, sobel_filter.T, mode="same"))
    edge = edge_x + edge_y
    edge = edge.reshape(img_array.shape)
    if images:
        fig, axs = plt.subplots(1, 2, figsize=(15, 5))
        axs[0].imshow(img)
        axs[0].set_title('Original Image')
        axs[1].imshow(edge, cmap="gray")
        axs[1].set_title("Line")
    for i in range(iterations):
        
        # Draw a random line on the processed image
        start_h, end_h, start_w, end_w = randomizeCoordinates(edge)
        start_point = (start_w, start_h)
        end_point = (end_w, end_h)


        # Create a boolean mask of the pixels where the edge intersects with the line
        mask = np.zeros_like(edge, dtype=np.uint8)
        cv2.line(mask, start_point, end_point, 255, thickness=1)
        edge = edge.astype(np.uint8)
        # Get the intersecting pixels between the edge image and the line mask
        intersecting_pixels = cv2.bitwise_and(edge, mask)
        intersecting_pixels[intersecting_pixels > 148] = 255

        # Display intersecting pixels and the line mask
        if images:
            fig, axs = plt.subplots(1, 2, figsize=(15, 5))
            axs[0].imshow(mask, cmap="gray")
            axs[0].set_title("Line")
            axs[1].imshow(intersecting_pixels, cmap='gray')
            axs[1].set_title('Intersecting Pixels')
        
        plt.show()

        # Save the modified image
        img_output = Image.fromarray(intersecting_pixels)
        img_output.save(f"output-{imgName}.jpg")

        newimg = Image.open(f'./output-{imgName}.jpg')
        newimg = newimg.convert("L")

        newimg_arr = np.array(newimg)
        cross_counter = np.sum(newimg_arr == 255)
        os.remove(f"output-{imgName}.jpg") 


        x = np.abs(end_h - start_h)
        y = np.abs(end_w - start_w)
        line_length = np.sqrt(x*x + y*y)
        #print(f"Iteration {i+1}:")
        if(cross_counter != 0):
            #print(line_length/cross_counter)
            values[i] = line_length/cross_counter
        else:
        #     print("No definitive intersections found")
            values[i] = 0
    
    print(f"{imgName}: \n Average mean of values from {iterations} iterations  = {np.mean(values)} [{pxToMicrometers(np.mean(values), scaleLength, unit)} um]\n")
    return np.mean(values)
with open("images.csv", "r") as imagesCSV:
    csvreader = csv.reader(imagesCSV)
    csvreader = next(csvreader)
    processes = []
    for row in csvreader:
        data = row[0].split(' ')
        print(data)
        process = Thread(target=calculate, args=(data[0],int(data[3]), int(data[4]),5000, False,))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

# ### TESTS ###
# def test1():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/1_A.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 12.5 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 16.3:
#         print("Test 1 PASSED")
#         return
#     print("Test 1 FAILED")

# def test2():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/1_B.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 11.4 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 13.4:
#         print("Test 2 PASSED")
#         return
#     print("Test 2 FAILED")

# def test3():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/2_A.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 11.9 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 15.4:
#         print("Test 3 PASSED")
#         return
#     print("Test 3 FAILED")
# def test4():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/2_B.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 13 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 13.8:
#         print("Test 4 PASSED")
#         return
#     print("Test 4 FAILED")
# def test5():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/3_A.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 13.8 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 16:
#         print("Test 5 PASSED")
#         return
#     print("Test 5 FAILED")
# def test6():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/3_B.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 15.7 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 16.7:
#         print("Test 6 PASSED")
#         return
#     print("Test 6 FAILED")
# def test7():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/4_A.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 14.2 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 15.6:
#         print("Test 7 PASSED")
#         return
#     print("Test 7 FAILED")
# def test8():
#     # image res [2048px x 1768px] => [74,61 um x 64,41 um]
#     img = Image.open('./tests/4_B.png')
#     gray = img.convert('L')
#     img_array = np.array(gray)
#     if 12.4 <= pxToMicrometers(calculate(img_array, 5000, False)) <= 14:
#         print("Test 8 PASSED")
#         return
#     print("Test 8 FAILED")
# test1()
# test2()
# test3()
# test4()
# test5()
# test6()
# test7()
# test8()


['1_A.png', '2048', '1768', '1098', '40']
['1_B.png', '2048', '1768', '1098', '40']
['2_A.png', '2048', '1768', '1098', '40']
['2_B.png', '2048', '1768', '1098', '40']
['3_A.png', '2048', '1768', '1098', '40']
['3_B.png', '2048', '1768', '1098', '40']
['4_A.png', '2048', '1768', '1098', '40']
['4_B.png', '2048', '1768', '1098', '40']
Average mean of values from 5000 iterations  = 360.09155316814685 [13.1180893686028 um]

Average mean of values from 5000 iterations  = 417.43983156398593 [15.207279838396573 um]

Average mean of values from 5000 iterations  = 378.9018221566129 [13.803345069457666 um]

Average mean of values from 5000 iterations  = 378.1406794998924 [13.775616739522494 um]

Average mean of values from 5000 iterations  = 408.21426930727233 [14.871193781685694 um]

Average mean of values from 5000 iterations  = 371.1389647755565 [13.520545164865446 um]

Average mean of values from 5000 iterations  = 404.9193274193285 [14.751159468827996 um]

Average mean of values from 5000 